In [1]:
import yaml
import pymongo
from urllib.parse import quote_plus as quote
import numpy as np
import random

collection_name = 'augmented_dataset' #['unique_dataset', 'augmented_dataset', 'initial_dataset']

In [2]:
# Подключимся к нашей коллекции

with open('../env/credsw.yaml', 'r') as file:
    creds_dict = yaml.safe_load(file)
    
url = 'mongodb://{user}:{pw}@{hosts}/?{rs}&authSource={auth_src}&{am}&tls=true&tlsCAFile={cert_file}'.format(
    user=creds_dict['username'],
    pw=quote(creds_dict['password']),
    hosts=creds_dict['host'],
    rs='replicaSet=rs01',
    auth_src=creds_dict['database'],
    am='authMechanism=DEFAULT',
    cert_file='../env/root.crt'
    )

dbs = pymongo.MongoClient(url)

db = dbs[creds_dict['database']]

collection = db[collection_name]

In [3]:
# Выгрузим данные из коллекции
pipeline = [{ "$unwind" : "$boxes" },
            {"$project":{
                "size": "$boxes.size",
                "stacking": "$boxes.stacking",
                "turnover": "$boxes.turnover",
                "loading_size": "$loading_size",
                "density_percent": "$density_percent"
                }},
            {"$group": {"_id": "$_id",
                        "loading_size": {"$first":"$loading_size"},
                        "density_percent": {"$first":"$density_percent"},
                        "boxes": {
                            "$push":  {
                                "size": "$size",
                                "stacking": "$stacking",
                                "turnover": "$turnover"                         
                            }
                            
                        }
            }
            }
            ]
result = collection.aggregate(pipeline, allowDiskUse=True)

In [4]:
# Подготовим train и test datasets
test_size = 1000

dataset = [
    (item['density_percent'],
    [[item['loading_size']['width'], item['loading_size']['height'], item['loading_size']['length'], False, False]] + \
    [ [box['size']['width'], box['size']['height'], box['size']['length'], box['stacking'], box['turnover']] for box in item['boxes']])
    for item in result
]

#y_train, y_test = [round(item[0],0) for item in dataset[0:-test_size]], [round(item[0],0) for item in dataset[-test_size : ]]
y_train, y_test = [[1 if i==round(item[0],0) else 0 for i in range(101)] for item in dataset[0:-test_size]], [[1 if i==round(item[0],0) else 0 for i in range(101)] for item in dataset[-test_size : ]]
X_train, X_test = [item[1] for item in dataset[0:-test_size]], [item[1] for item in dataset[-test_size:]]

In [5]:
# Выровняем размерности входных матриц X
max_size = max(max([len(i) for i in X_train]), max([len(i) for i in X_test]))

for i in range(len(X_train)):
    add_dims = max_size - len(X_train[i])
    X_train[i] = np.concatenate((np.array(X_train[i]), np.zeros((add_dims, 5))))

for i in range(len(X_test)):
    add_dims = max_size - len(X_test[i])
    X_test[i] = np.concatenate((np.array(X_test[i]), np.zeros((add_dims, 5))))


# LeNet-5-like CNN

In [6]:
from keras.models import Sequential
from keras.layers import Conv1D, AveragePooling1D, Flatten, Dense
from keras.callbacks import ModelCheckpoint

In [7]:
model = Sequential()

model.add(Conv1D(filters=6, kernel_size=2, strides=1, padding='same', activation='tanh', input_shape=(max_size, 5)))
model.add(AveragePooling1D(pool_size=2, strides=2, padding='valid'))

model.add(Conv1D(filters=16, kernel_size=2, strides=1, padding='valid', activation='tanh'))
model.add(AveragePooling1D(pool_size=2, strides=2, padding='valid'))

model.add(Conv1D(filters=120, kernel_size=2, strides=1, padding='valid', activation='tanh'))

model.add(Flatten())

model.add(Dense(900, activation='tanh'))

model.add(Dense(101, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2850, 6)           66        
                                                                 
 average_pooling1d (AverageP  (None, 1425, 6)          0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, 1424, 16)          208       
                                                                 
 average_pooling1d_1 (Averag  (None, 712, 16)          0         
 ePooling1D)                                                     
                                                                 
 conv1d_2 (Conv1D)           (None, 711, 120)          3960      
                                                                 
 flatten (Flatten)           (None, 85320)             0

In [8]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [9]:
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)
hist = model.fit(np.array(X_train),np.array(y_train), batch_size=50, epochs=20, validation_data=(np.array(X_test), np.array(y_test)),callbacks=[checkpointer], verbose=2, shuffle=True)

Epoch 1/20

Epoch 1: val_loss improved from inf to 1.87864, saving model to model.weights.best.hdf5
187/187 - 66s - loss: 1.9608 - accuracy: 0.3258 - val_loss: 1.8786 - val_accuracy: 0.3970 - 66s/epoch - 352ms/step
Epoch 2/20

Epoch 2: val_loss improved from 1.87864 to 1.52410, saving model to model.weights.best.hdf5
187/187 - 69s - loss: 1.4120 - accuracy: 0.5217 - val_loss: 1.5241 - val_accuracy: 0.4080 - 69s/epoch - 369ms/step
Epoch 3/20

Epoch 3: val_loss improved from 1.52410 to 1.20693, saving model to model.weights.best.hdf5
187/187 - 75s - loss: 1.1679 - accuracy: 0.6173 - val_loss: 1.2069 - val_accuracy: 0.6160 - 75s/epoch - 400ms/step
Epoch 4/20

Epoch 4: val_loss improved from 1.20693 to 1.00502, saving model to model.weights.best.hdf5
187/187 - 91s - loss: 0.9858 - accuracy: 0.6810 - val_loss: 1.0050 - val_accuracy: 0.7010 - 91s/epoch - 484ms/step
Epoch 5/20

Epoch 5: val_loss improved from 1.00502 to 0.88471, saving model to model.weights.best.hdf5
187/187 - 147s - loss: 0

In [10]:
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)

In [11]:
print(f"Test accuracy: {score[1]}")

Test accuracy: 0.8899999856948853


### Проверка на нескольких случайных примерах из test set

In [12]:
not_guessed = list()
for i in random.sample(range(1000), 1000):
    prediction = model.predict(np.array([X_test[i]]), verbose=0)
    prediction = np.where(prediction[0]==max(prediction[0]))[0][0]
    actual = y_test[i].index(1)
    print(f"Prediction: {prediction}; Actual: {actual}; Guessed: {actual==prediction}")
    if actual!=prediction:
        not_guessed.append(X_test[i])

Prediction: 70; Actual: 70; Guessed: True
Prediction: 83; Actual: 83; Guessed: True
Prediction: 83; Actual: 83; Guessed: True
Prediction: 100; Actual: 100; Guessed: True
Prediction: 100; Actual: 100; Guessed: True
Prediction: 83; Actual: 83; Guessed: True
Prediction: 87; Actual: 87; Guessed: True
Prediction: 70; Actual: 70; Guessed: True
Prediction: 96; Actual: 62; Guessed: False
Prediction: 70; Actual: 70; Guessed: True
Prediction: 74; Actual: 74; Guessed: True
Prediction: 70; Actual: 70; Guessed: True
Prediction: 100; Actual: 100; Guessed: True
Prediction: 83; Actual: 83; Guessed: True
Prediction: 87; Actual: 87; Guessed: True
Prediction: 74; Actual: 74; Guessed: True
Prediction: 100; Actual: 100; Guessed: True
Prediction: 100; Actual: 100; Guessed: True
Prediction: 70; Actual: 70; Guessed: True
Prediction: 100; Actual: 100; Guessed: True
Prediction: 100; Actual: 100; Guessed: True
Prediction: 83; Actual: 83; Guessed: True
Prediction: 70; Actual: 70; Guessed: True
Prediction: 83; Act

In [13]:
lengths = [sum([sum(x)>0 for x in not_guessed[i]]) for i in range(len(not_guessed))]

In [14]:
lengths.sort()

In [16]:
len(lengths)

110